# AutoML Forecasting in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a Forecasting AutoML task in pipeline.

**Motivations** - This notebook explains how to use Forecasting AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, command, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.automl import forecasting
from azure.ai.ml.entities._job.automl.tabular.forecasting_settings import ForecastingSettings

# add environment variable to enable private preview feature
import os
os.environ["AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED"] = "true"

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Basic pipeline job with Classification task

## 2.1 Build pipeline

In [ ]:
# Define pipeline
@pipeline(
    description="AutoML Forecasting Pipeline",
    )
def automl_forecasting(
    forecasting_train_data,
):
    # define forecasting settings
    forecasting_settings = ForecastingSettings(time_column_name="timeStamp", forecast_horizon=12, frequency="H", target_lags=[ 12 ], target_rolling_window_size=4)
    
    # define the automl forecasting task with automl function
    forecasting_node = forecasting(
        training_data=forecasting_train_data,
        target_column_name="demand",
        primary_metric="normalized_root_mean_squared_error",
        n_cross_validations=2,
        forecasting_settings=forecasting_settings,
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    
    forecasting_node.set_limits(timeout_minutes=180)

    # define command function for registering the model
    command_func = command(
        inputs=dict(
            model_input_path = Input(type="mlflow_model"),
            model_base_name= "forecasting_example_model"
        ),
        code="./register.py",
        command="python register.py " +
                "--model_input_path ${{inputs.model_input_path}} " +
                "--model_base_name ${{inputs.model_base_name}}" ,

        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )
    register_model = command_func(model_input_path = forecasting_node.outputs.best_model)

    # save the configuration file
    deployment_config_path = "deployment_configuration.json"
    
    # define command function for deploying the model
    deploy_command_func = command(
        inputs=dict(
            deployment_config_path = Input(type="str"),
            model_base_name= "forecasting_example_model"
        ),
        
        code="./deploy.py",
        command="python deploy.py " +
                "--deployment_config_path ${{inputs.deployment_config_path}} " +
                "--model_base_name ${{inputs.model_base_name}}",

        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )
    deploy_model = deploy_command_func(deployment_config_path = deployment_config_path)

data_folder = "./data"
pipeline = automl_forecasting(
    forecasting_train_data=Input(path=f"{data_folder}/training-mltable-folder/", type="mltable"),
)

# set pipeline level compute
pipeline.settings.default_compute="cpu-cluster"
        

# 2.2 Submit pipeline job

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)